In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import logging
from tqdm import tqdm
import pandas as pd
from toolkit import simplex, link_prediction
from toolkit.utils import hyperedges_to_edges

logging.basicConfig(level=logging.INFO)

/home/gergo/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def read_zabka_csv(csv_path):
    df = pd.read_csv(csv_path)
    nodes = []
    hyper_edges = []
    edges = []
    logging.info(f' Reading baskets from zabka dataset')
    for order_id in tqdm(df["basket"].drop_duplicates()):
        basket = df[df["basket"] == order_id]["product_id"].values.tolist()
        hyper_edges.append(basket)
        nodes += basket
        edges += [(x, y) for x in basket for y in basket if x < y]
    nodes = list(set(nodes))
    edges = list(set(edges))
    return len(nodes), edges, hyper_edges

In [4]:
#(nodes_num, edges, hyperedges) = read_zabka_csv('~/Downloads/zabka.csv')

In [5]:
%store -r hyperedges

In [6]:
s = simplex.SimplicalComplex(hyperedges)

INFO:root:5
100%|██████████| 210408/210408 [00:14<00:00, 14384.11it/s]
/home/gergo/Documents/python/simplical_complex/toolkit/simplex.py:265: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at  ../aten/src/ATen/SparseCsrTensorImpl.cpp:68.)
  sparse_csr = torch.sparse_csr_tensor(
INFO:root: Generating embedding with 1 depth
INFO:root: Creating dense matrix of size: rows: 185, cols: 185.
INFO:root: Generating embedding with 3 depth
INFO:root: Creating dense matrix of size: rows: 189, cols: 189.
INFO:root: Creating dense matrix of size: rows: 189, cols: 189.
INFO:root: It took 0.01392364501953125 to compute this product (1).
INFO:root: Creating dense matrix of size: rows: 21, cols: 21.
INFO:root: Creating dense matrix of size: rows: 21, cols: 21.
INFO:root: It took 0.0028252601623535156 to compute this product (1).


In [ ]:
graph_embedding = simplex.generate_embedding(s, 1)
# here we just want to get the max depth, so set it unrealistically high
hyeprgraph_embedding = simplex.generate_embedding(s, 10)

In [7]:
edges = hyperedges_to_edges(s.hypergraph)

In [8]:
#need to remap the edges
edges_remapped = [
        tuple([s.node_persistor[simplex.Label([v])] for v in edge])
    for edge in edges
]

In [9]:
(g, link_prediction_metric) = link_prediction.link_prediction_setup(
    len(s.node_persistor),
    edges_remapped
)

In [10]:
link_prediction_metric(graph_embedding)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values